# LSTM Model Experiments

In [1]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

from collections import Counter
from sklearn.metrics import classification_report

In [2]:
MAX_VOCAB_SIZE = 20000  # Taille du vocabulaire max
MAX_LEN = 256           # Longueur max des séquences
EMBED_DIM = 100         # Dimension des embeddings
HIDDEN_DIM = 128        # Taille des couches LSTM
BATCH_SIZE = 16         # Taille des batchs
EPOCHS = 5              # Nombre d'époques

In [3]:
# Load datasets
train_df = pd.read_csv("data/kaggle/preprocessed/train.csv")
test_df = pd.read_csv("data/kaggle/preprocessed/test.csv")

In [9]:
def tokenize(text):
    return text.split()

def encode(vocab, text):
    return [vocab.get(tok, 1) for tok in tokenize(text)[:MAX_LEN]]

In [10]:
# Build vocabulary
counter = Counter()
for text in train_df["text"]:
    tokens = tokenize(text)
    counter.update(tokens)

most_common = counter.most_common(MAX_VOCAB_SIZE - 2)
vocab = {"<PAD>": 0, "<UNK>": 1}
for i, (word, _) in enumerate(most_common, start=2):
    vocab[word] = i

In [11]:
# Custom Dataset
class TextDataset(Dataset):
    def __init__(self, df):
        self.texts = [torch.tensor(encode(vocab, text), dtype=torch.long) for text in df["text"]]
        self.labels = torch.tensor(df["label"].values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [12]:
def collate_fn(batch):
    texts, labels = zip(*batch)
    texts = pad_sequence(texts, batch_first=True, padding_value=0)
    texts = texts[:, :MAX_LEN]  # truncate if needed
    return texts, torch.tensor(labels)

In [13]:
# DataLoaders
train_ds = TextDataset(train_df)
test_ds = TextDataset(test_df)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=32, collate_fn=collate_fn)

In [14]:
# LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab["<PAD>"])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Initialize model
model = LSTMClassifier(len(vocab), embedding_dim=100, hidden_dim=128, output_dim=len(train_df["label"].unique()))
model = model.to(device)

# Training
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(100):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

Using device: cpu


In [16]:
# Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y_batch.numpy())

print(classification_report(all_labels, all_preds))


              precision    recall  f1-score   support

           0       0.61      0.54      0.57       624
           1       0.60      0.66      0.63       643

    accuracy                           0.60      1267
   macro avg       0.60      0.60      0.60      1267
weighted avg       0.60      0.60      0.60      1267

